In [1]:
import sys
sys.path.append("..")

import ecole
import random
import operator
import numpy as np
from pathlib import Path

from models.generator.albp_gen import ALBPGenerator

from deap import algorithms, base, creator, tools, gp

In [2]:
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

# define the primitive set for the GP algorithm
NUM_VAR_FEATURES = 19
pset = gp.PrimitiveSet("MAIN", NUM_VAR_FEATURES)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(min, 2)
pset.addPrimitive(max, 2)

# define the problem
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

instances = ALBPGenerator(
    directory=Path("..", "data", "raw"),
    rng=np.random.default_rng(430958)
)

scip_parameters = {
    "separating/maxrounds": 0,
    "presolving/maxrestarts": 0,
    "limits/time": 3600,
}

env = ecole.environment.Branching(
    observation_function=ecole.observation.NodeBipartite(),
    scip_params=scip_parameters,
)

N_EVAL = 10

# define an evaluation function, which MUST return a tuple
def evalSymbReg(individual):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # pick a random instance
    scores = []
    evaluations = 0
    while evaluations < N_EVAL:
        instance = next(instances)
        observation, action_set, _, done, _ = env.reset(instance)
        while not done:
            b = map(func, observation.variable_features)
            action = np.argmax(b)
            observation, action_set, _, done, _ = env.step(action_set[action])
        scores.append(env.model.primal_bound)


    return sum(scores) / len(scores),

# register the fitness function
toolbox.register("evaluate", evalSymbReg)
# register the selection operator: a tournament operator of size 3
toolbox.register("select", tools.selTournament, tournsize=3)
# add mutation operators
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

def main():
    random.seed(318)

    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop, toolbox, 0.5, 0.1, 40, stats=mstats,
                                   halloffame=hof, verbose=True)
    # print log
    return pop, log, hof


In [3]:
main()

original problem has 903073 variables (903073 bin, 0 int, 0 impl, 0 cont) and 3415 constraints


: 

In [ ]:
# show results
